In [31]:
from plotly_calplot import calplot
import pandas as pd
import nbformat
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

# リサーチマップのアクセス解析

In [37]:
df = pd.read_csv("access_data/researchmap.csv")
df["value"] = df["value"].astype(int)
df['date'] = pd.to_datetime(df['date'])
df.columns

Index(['date', 'value'], dtype='object')

In [38]:
custom_colorscale = [
    [0.0, "#3f3f3f"],  # スケールの最小値に対応する色
    [1.0, "#00d5ff"]    # スケールの最大値に対応する色
]

# タイルプロット
fig = calplot(df, x="date", y="value",
              colorscale=custom_colorscale)
fig.update_layout({"yaxis": {"title": "2024"},
                  "plot_bgcolor": "#3f3f3f",
                  "paper_bgcolor": "#3f3f3f"})
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.write_html("html_figures/researchmap.html")
fig

# 個人ページのアクセス解析

In [40]:
df = pd.read_csv("access_data/personal.csv")
df["value"] = df["value"].astype(int)
df['date'] = pd.to_datetime(df['date'])
df.columns

Index(['date', 'value'], dtype='object')

In [41]:
custom_colorscale = [
    [0, "#3f3f3f"],  # スケールの最小値に対応する色
    [1, "#fff700"],       # 中間値に対応する色
]

# タイルプロット
fig = calplot(df, x="date", y="value",
              colorscale=custom_colorscale)
fig.update_layout({"yaxis": {"title": "2024"},
                  "plot_bgcolor": "#3f3f3f",
                  "paper_bgcolor": "#3f3f3f"})
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.write_html("html_figures/personal.html")
fig

# リサーチマップと個人サイトのアクセス数推移

In [42]:
start_date = '2024-11-26'
end_date = '2025-11-26'

# リサーチマップデータ
df_rm = pd.read_csv("access_data/researchmap.csv")
df_rm['date'] = pd.to_datetime(df_rm['date'])
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_date = pd.DataFrame(date_range, columns=['date'])
df_rm = pd.merge(df_date, df_rm, on='date', how='left')
df_rm["category"] = "researchmap"


# このサイトのデータ
df_p = pd.read_csv("access_data/personal.csv")
df_p['date'] = pd.to_datetime(df_p['date'])
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_date = pd.DataFrame(date_range, columns=['date'])
df_p = pd.merge(df_date, df_p, on='date', how='left')
df_p["category"] = "this page"

# 2つのでデータを連結
df = pd.concat([df_rm, df_p])
df

,date,value,category
0,2024-11-26,18.0,researchmap
1,2024-11-27,4.0,researchmap
2,2024-11-28,2.0,researchmap
3,2024-11-29,2.0,researchmap
4,2024-11-30,2.0,researchmap
...,...,...,...
361,2025-11-22,NaN,this page
362,2025-11-23,NaN,this page
363,2025-11-24,NaN,this page
364,2025-11-25,NaN,this page


In [43]:
line = px.line(df, x='date', y='value', color = "category",
        color_discrete_map={'researchmap': '#00d5ff', 'this page': '#f2ff00'})
line.update_layout(margin=dict(l=10, r=10, t=10, b=10))
line.update_layout(
    legend=dict(
        x=0,  # x位置（0で左端）
        y=1,  # y位置（1で上端）
        traceorder='normal',  # 凡例アイテムの順番
        orientation='v'  # 縦方向で配置
    )
)
line.update_layout(
    yaxis=dict(
        showgrid=True,  # y軸の目盛り線を表示
        showticklabels=True,  # y軸の目盛りラベルを表示
    )
)
line.update_layout(
    plot_bgcolor="white",  # 描画領域の背景色を白
    paper_bgcolor="white",  # グラフ全体の背景色を白
    xaxis=dict(
        showgrid=True,  # 目盛り線を表示
        gridcolor="gray",  # x軸の目盛り線をグレーに設定
    ),
    yaxis=dict(
        showgrid=True,  # 目盛り線を表示
        gridcolor="gray",  # y軸の目盛り線をグレーに設定
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        orientation='v'
    )
)

line.update_layout({"plot_bgcolor": "#3f3f3f",
                    "paper_bgcolor": "#3f3f3f"})

line.update_layout(
    xaxis_title='',  # x軸のラベルを空に
    yaxis_title=''   # y軸のラベルを空に
)

line

In [44]:
line.write_html("html_figures/lineplot.html")

# 地図を描こう
### アクセスされているけど，まだ緯度経度を登録していない地域を検出

In [45]:
rm_region = pd.read_csv("access_data/researchmap_region.csv")
p_region = pd.read_csv("access_data/personal_region.csv")
region = pd.concat([rm_region, p_region]).groupby(["地域", "市区町村"]).sum().reset_index()
region_pref_muni = region[["地域", "市区町村"]].rename(columns = {"地域":"pref", "市区町村":"city"})
#region_pref_muni.to_csv("access_data/region_pref_muni.csv")
region_pref_muni

,pref,city
0,(not set),(not set)
1,(not set),Hong Kong
2,(not set),Singapore
3,Aichi,Kariya
4,Aichi,Kasugai
...,...,...
186,Wakayama,Shirahama
187,Washington,Mattawa
188,Yamagata,Yamagata
189,Yamanashi,Kofu


In [46]:
coords = pd.read_csv("access_data/lon_lat.csv")
coords_region = pd.merge(region_pref_muni, coords, how="outer")
coords_region = coords_region[coords_region["lon"].isna() == True]
coords_region[["pref", "city"]].to_csv("access_data/dont_have_lon_lat.csv")
coords_region
# まだ座標を当たられていない場所を取得

,pref,city,lat,lon
1,(not set),Hong Kong,NaN,NaN
2,(not set),Singapore,NaN,NaN
30,Colorado,(not set),NaN,NaN
64,Ibaraki,(not set),NaN,NaN
88,Kanchanaburi,(not set),NaN,NaN
97,Masovian Voivodeship,(not set),NaN,NaN
120,Ontario,(not set),NaN,NaN
144,Taipei City,(not set),NaN,NaN
152,Tokyo,(not set),NaN,NaN


### リサーチマップ

In [47]:
# 座標の読み込み
coords = pd.read_csv("access_data/lon_lat.csv")

# rmアクセス数の読み込み
rm_region = pd.read_csv("access_data/researchmap_region.csv")
rm_region["city"] = rm_region["市区町村"]
rm_region["active_user"] = rm_region["アクティブ ユーザー"]
rm_df = rm_region.merge(coords)
rm_df["cite"] = "researchmap"
rm_df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),450,443,219,0.452479,0.486667,17.415556,1860,0,0,0,(not set),450,(not set),35.680929,139.763317,researchmap
1,Miyagi,Sendai,55,53,104,0.712329,1.890909,107.418182,1294,0,0,0,Sendai,55,Miyagi,38.268050,140.872263,researchmap
2,Tokyo,Shinjuku City,37,35,12,0.307692,0.324324,8.054054,129,0,0,0,Shinjuku City,37,Tokyo,35.693776,139.703742,researchmap
3,Osaka,Osaka,32,31,14,0.437500,0.437500,10.125000,127,0,0,0,Osaka,32,Osaka,34.700723,135.495872,researchmap
4,Tokyo,Arakawa City,19,18,5,0.263158,0.263158,6.736842,65,0,0,0,Arakawa City,19,Tokyo,35.736103,139.783594,researchmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Washington,Mattawa,1,1,1,1.000000,1.000000,17.000000,10,0,0,0,Mattawa,1,Washington,46.737354,-119.901469,researchmap
170,Yamagata,Yamagata,1,1,1,1.000000,1.000000,11.000000,4,0,0,0,Yamagata,1,Yamagata,38.248281,140.327441,researchmap
171,Zurich,Zurich,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Zurich,1,Zurich,47.377948,8.536401,researchmap
172,Hokkaido,Asahikawa,0,0,0,0.000000,0.000000,0.000000,2,0,0,0,Asahikawa,0,Hokkaido,43.762513,142.358775,researchmap


### 個人サイト

In [48]:
# 座標の読み込み
coords = pd.read_csv("access_data/lon_lat.csv")

# rmアクセス数の読み込み
p_region = pd.read_csv("access_data/personal_region.csv")
p_region["city"] = p_region["市区町村"]
p_region["active_user"] = p_region["アクティブ ユーザー"]
p_df = p_region.merge(coords)
p_df["cite"] = "personal"
p_df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),54,51,44,0.523810,0.814815,13.50,295,0,0,0,(not set),54,(not set),35.680929,139.763317,personal
1,Miyagi,Sendai,20,19,157,0.758454,7.850000,228.85,1077,0,0,0,Sendai,20,Miyagi,38.268050,140.872263,personal
2,(not set),(not set),13,13,1,0.076923,0.076923,4.00,50,0,0,0,(not set),13,(not set),35.680929,139.763317,personal
3,Colorado,(not set),10,10,0,0.000000,0.000000,0.00,30,0,0,0,(not set),10,(not set),35.680929,139.763317,personal
4,Masovian Voivodeship,(not set),5,5,0,0.000000,0.000000,0.00,15,0,0,0,(not set),5,(not set),35.680929,139.763317,personal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Tokyo,Nerima City,1,1,1,1.000000,1.000000,9.00,4,0,0,0,Nerima City,1,Tokyo,35.746888,139.617910,personal
57,Tokyo,Shibuya,1,0,0,0.000000,0.000000,1.00,3,0,0,0,Shibuya,1,Tokyo,35.669436,139.701884,personal
58,Tokyo,Tama,1,1,1,1.000000,1.000000,27.00,5,0,0,0,Tama,1,Tokyo,35.637111,139.446856,personal
59,Yamanashi,Kofu,1,1,0,0.000000,0.000000,0.00,3,0,0,0,Kofu,1,Yamanashi,35.660131,138.575896,personal


In [49]:
df = pd.concat([rm_df, p_df])
df

,地域,市区町村,アクティブ ユーザー,新規ユーザー数,エンゲージのあったセッション数,エンゲージメント率,エンゲージメントのあったセッション数（1 アクティブ ユーザーあたり）,アクティブ ユーザーあたりの平均エンゲージメント時間,イベント数,キーイベント,ユーザー キーイベント率,合計収益,city,active_user,pref,lat,lon,cite
0,Tokyo,(not set),450,443,219,0.452479,0.486667,17.415556,1860,0,0,0,(not set),450,(not set),35.680929,139.763317,researchmap
1,Miyagi,Sendai,55,53,104,0.712329,1.890909,107.418182,1294,0,0,0,Sendai,55,Miyagi,38.268050,140.872263,researchmap
2,Tokyo,Shinjuku City,37,35,12,0.307692,0.324324,8.054054,129,0,0,0,Shinjuku City,37,Tokyo,35.693776,139.703742,researchmap
3,Osaka,Osaka,32,31,14,0.437500,0.437500,10.125000,127,0,0,0,Osaka,32,Osaka,34.700723,135.495872,researchmap
4,Tokyo,Arakawa City,19,18,5,0.263158,0.263158,6.736842,65,0,0,0,Arakawa City,19,Tokyo,35.736103,139.783594,researchmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Tokyo,Nerima City,1,1,1,1.000000,1.000000,9.000000,4,0,0,0,Nerima City,1,Tokyo,35.746888,139.617910,personal
57,Tokyo,Shibuya,1,0,0,0.000000,0.000000,1.000000,3,0,0,0,Shibuya,1,Tokyo,35.669436,139.701884,personal
58,Tokyo,Tama,1,1,1,1.000000,1.000000,27.000000,5,0,0,0,Tama,1,Tokyo,35.637111,139.446856,personal
59,Yamanashi,Kofu,1,1,0,0.000000,0.000000,0.000000,3,0,0,0,Kofu,1,Yamanashi,35.660131,138.575896,personal


In [50]:
fig = px.scatter_mapbox(
    df,
    lat='lat',  # 緯度
    lon='lon',  # 経度
    size='active_user',
    center={'lat': 37.076045, 'lon': 136.971889},
    zoom=4,
    size_max=40,
    color_discrete_sequence=["#00d5ff", "#fff700"],
    color="cite"  # サイト別に色分け
)

fig.update_layout(
    title_font_size=20,
    geo=dict(
        showcountries=True,  # 国境を表示
        showcoastlines=True,  # 海岸線を表示
        coastlinecolor="Gray",
        landcolor="LightYellow"
    ),
    mapbox_style='carto-darkmatter',
    margin=dict(l=0, r=0, t=0, b=0),
    legend=dict(
        x=1,  # 凡例を右に移動
        y=1,  # 凡例を上に移動
        xanchor="right",  # 凡例の右端を基準に配置
        yanchor="top",  # 凡例の上端を基準に配置
        bgcolor="rgba(255, 255, 255, 0.7)",  # 背景を半透明の白に設定
        bordercolor="Black",  # 枠線の色
        borderwidth=1  # 枠線の幅
    )
)

fig.write_html("html_figures/access_map.html")
fig.show()
